In [2]:
import codebase.web_scrape_functions as wsf
import codebase.analysis_functions as af
import codebase.match_data as match_data
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import utils
from utils import logger
import os
import logging
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import codebase.graphing_functions as gf
from codebase.settings import CAREERS
from codebase.settings import LABEL_DATA

if utils.check_if_ipython():
    logger.disabled = True

%load_ext autoreload
%autoreload 2
logger.handlers[1].setLevel(logging.INFO)
# logger.disabled = True
pd.get_option("display.max_columns")

20

First step is to get Kohli's career averges for all shots, then his averages for just specific shots and then we can find his most productive shot. The problem with searching things based off of specific shots strings only is that there is a veriety in the way that shots are described in the commentary. We would need a machine learning algorithm to work and recognize what these shots are. But first we need to create a training set which is a pain in the ass.

In [3]:
KOHLI_ID = '253802'
ROOT_PLAYER_ID = '303669'
WILLIAMSON_PLAYER_ID = '277906'
SPD_SMITH_ID = '267192'

In [4]:
kohli_matches = wsf.get_player_match_list(KOHLI_ID)

In [5]:
kohli_innings = af.get_cricket_totals(KOHLI_ID, kohli_matches, _type='bat', by_innings=True, is_object_id=True, from_scorecards=False)

In [8]:
kohli_comms = af.get_player_contributions(KOHLI_ID, kohli_matches, _type='bat', by_innings=True, is_object_id=True)

In [9]:
kohli_comms_flat = [x for comms in kohli_comms for x in comms.commentTextItems.to_list()]

In [10]:
def search_shots_in_comms(contributions, search_keywords, exlude_words=[], primary_keywords=[], threshold = 0.5):
    shots = []
    for i, comms in enumerate(contributions):
        innings = comms.commentTextItems.to_list()
        search = af.search_for_phrases(innings, keywords=search_keywords, exclude_words=exlude_words, primary_keywords=primary_keywords, threshold = threshold)
        shots.append(comms.iloc[[i for i,x in enumerate(search[0]) if x == 1]])
        
    return shots

In [ ]:
## Cover drives 
cover_drive_primary_kw = ['cover drive', 'cover-drive']

cover_drive_search_kw = [
    'drive',
    ('punch', 0.3),
    ('cover', 0.3),
    ('full', 0.2),
    ('wide', 0.2) ,
    'outside edge',
    ('reach', 0.3),
    'slip',
    'edge',
    'driving'
]
cover_drive_exlude_words = [
    'run out',
    'pull', 'flick',
    'bouncer', 'short ball', 'stays back',
    'backfoot', 'top edge', 'leave',
    'lets one go', 'easy leave', 'leaves the ball',
    'leg side', 'leading edge', 'leg ', 'cut', 'leg.',
    'on the pads', 'left alone', ('short', 0.25)
]
cover_drives_played = search_shots_in_comms(kohli_comms, cover_drive_search_kw, cover_drive_exlude_words, primary_keywords=cover_drive_primary_kw)

In [ ]:
cover_drive_primary_kw = ['cover drive', 'cover-drive']

cover_drive_search_kw = [
    'drive',
    ('punch', 0.3),
    ('cover', 0.3),
    ('off side', 0.2),
    ('full', 0.25),
    ('wide', 0.2) ,
    ('outside edge', 0.3),
    ('reach', 0.3),
    ('slip', 0.25),
    ('edge', 0.3),
    'driving',
    ('force', 0.2),
    ('push', 0.3),
    ('pitched up', 0.3),
    ('Overpitched', 0.2),
    ('dab', 0.2),
    ('outside off', 0.25)

]
cover_drive_exlude_words = [
    'run out',
    'pull', 'flick',
    'bouncer', 'short ball', 'stays back',
    'backfoot', 'top edge', 'leave', 'back foot', ('back', 0.2),
    'lets one go', 'easy leave', 'leaves the ball', 'goes back',
    'leg side', 'leading edge', ('leg ', 0.25), 'cut', ('leg', 0.25),
    'on the pads', 'left alone', ('short', 0.25), ('midwicket', 0.2), ('long-on', 0.25), ('long on', 0.25), ('back of a length', 0.25)
]



In [ ]:
ind, scores = af.search_for_phrases(kohli_comms_flat, cover_drive_search_kw, cover_drive_exlude_words, primary_keywords=cover_drive_primary_kw, threshold=0.49)
cover_drives_career = [kohli_comms_flat[i] for i,x in enumerate(ind) if x==1]
cover_drives_almost = [kohli_comms_flat[i] for i,x in enumerate(scores) if x < 0.49 and x > 0]

In [ ]:
print(len(cover_drives_career))
cover_drives_career

In [ ]:
print(len(cover_drives_almost))
cover_drives_almost[1000:]

In [ ]:
cover_drive_averages, cover_drive_totals, cover_drive_stats = af.analyse_batting(cover_drives_played)

In [ ]:
cover_drive_averages

In [ ]:
cover_drive_totals

In [ ]:
flick_primary_kw = [
    'flick'
]
flick_search_kw = [
    'tuck',
    'clipped',
    'clip',
    ('pads', 0.3),
    ('pad ', 0.3),
    'on the pads',
    #('leg side', 0.25),
    ('leg', 0.3),
    ('square', 0.19),
    'glance'
    'off the pads',
    'off his pads',
    ('nudge', 0.3),
    ('pick', 0.3),
    ('straight', 0.3),
    #('square leg', 0.25),
    ('whip', 0.4),
    'on the legs',
    ('work', 0.25),
    ('midwicket', 0.3),
    ('on side', 0.25),
    ('across', 0.3)
]
flick_exlude_words = [
    'run out', 'bumper', 'sweep', 'swept',
    'pull', 'drive', 'cover-drive', ('cover', 0.3),
    'bouncer', 'short ball', 'stays back',
    'backfoot', 'top edge',
    'lets one go', 'easy leave', 'leaves the ball', 'padded away',
    ('off side', 0.25), 'cut', 'left alone', ('point', 0.25), ('defend', 0.3), ('push', 0.25)
]

flick_shots_career = search_shots_in_comms(kohli_comms, flick_search_kw, flick_exlude_words, flick_primary_kw, threshold=0.49)

In [ ]:
flick_averages, flick_totals, flick_stats = af.analyse_batting(flick_shots_career)

In [ ]:
flick_averages
#Flicks mostly produce singles from Kohli, there are almost as many ones as there are dots with flicks, so this is a genrally 'safe' shot for Kohli which is quite productive.

In [ ]:
flick_totals

In [44]:
pull_primary_kw = ['short-arm jab']

pull_search_kw = [
    'pull',
    'hook',
    ('short', 0.2),
    ('swivel', 0.3),
    ('bouncer', 0.2),
    ('bumper', 0.2),
    ('hammer', 0.3),
    ('slam',  0.3),
    ('slap',  0.3),
    ('thump', 0.3),
    ('smash', 0.3),
    ('pummels', 0.3)

]
pull_exlude_words = [
    'full',
    'cut',
    'flick',
    'drive',
    'push',
    'block',
    'outside edge',
    ('cover', 0.3),
    ('pads', 0.3), ('off side', 0.25), ('point', 0.25),
    'lets one go', 'easy leave', 'leaves the ball', 'left alone', 'lets it go', 'inside edge'
    
]

In [45]:
pull_ind, pull_scores = af.search_for_phrases(kohli_comms_flat, pull_search_kw, pull_exlude_words, primary_keywords=pull_primary_kw, threshold=0.49)
pulls_career = [(kohli_comms_flat[i], i) for i,x in enumerate(pull_ind) if x==1]
pulls_almost = [(kohli_comms_flat[i], i) for i,x in enumerate(pull_scores) if x < 0.49 and x > 0]

In [46]:
pulls_career

[('Long hop to end the over, kohli latches on with the pull but edwards does well to dive at midwicket and save the boundary.',
  138),
 ('Pulled away to deep square, retains strike', 244),
 ("First boundary since sehwag's dismissal. this is a long hop, asking to be hit,. kohli obliges by going back and pulling it over midwicket. which also shows india haven't gone into a shell. they are cautious but still awake to rank bad deliveries",
  364),
 ("Short ball, across him, kohli pulls it, wide of mid-on, but can't find the gap",
  397),
 ("A long hop, and he has pulled this smack into barath's shoulder at forward short leg. that could have gone for four. that ricochet could have lobbed for a catch. to borrow from bill lawry, it's all happening",
  400),
 ("Misfield. short of a length, pulled away from outside off, through bishoo at short midwicket. didn't go down properly. that injury must bothering him",
  413),
 ('Too short, kohli goes deep into the crease to pull this along the ground

In [47]:
pulls_almost

[('There arrives the short ball, but a touch outside off stump and virat calmly weaves out of its path',
  6),
 ('Pitches leg, turns away, but it slightly short, and kohli deals easily off the back foot',
  31),
 ('Bouncer, too short really, easy for kohli to duck under', 34),
 ('Good bouncer, in the chest-throat area, fended past short leg', 41),
 ('This bouncer bounces as high as expected, and he can duck under it', 43),
 ("Wasted bouncer, down the leg side, easy to stay out of harm's way here",
  44),
 ('Short, kohli fends with the bottom hand off, straight into the ground. more staring',
  45),
 ("Oversteps striving for extra once, as it turns out this is too short and kohli easily ducks under it. actually no-ball called from square leg because there were three fielders behind square on the on side. two fielders in the deep and barath's one foot was behind the crease at short leg",
  46),
 ("That's poorly judged by barath at short leg. fidel has kohli fending, the ball lobs and fal

In [48]:
pull_shots_career = search_shots_in_comms(kohli_comms, pull_search_kw, pull_exlude_words, primary_keywords=pull_primary_kw, threshold=0.49)
pull_averages, pull_totals, pull_stats = af.analyse_batting(pull_shots_career)

In [49]:
pull_averages

{'runs': 6.33,
 'dismissals': 0.05,
 'balls': 3.53,
 'sr': 1.73,
 'average': nan,
 'dot_balls': 1.0,
 'ones': 1.15,
 'twos': 0.27,
 'threes': 0.03,
 'fours': 1.03,
 'fives': 0.0,
 'sixes': 0.07,
 'total_balls_faced': 97.82,
 'fours_per_ball': 0.29,
 'sixes_per_ball': 0.01,
 'dots_per_ball': 0.32}

In [50]:
pull_totals

{'runs': 620,
 'dismissals': 5,
 'balls': 346,
 'sr': 169.6,
 'average': nan,
 'dot_balls': 98,
 'ones': 113,
 'twos': 26,
 'threes': 3,
 'fours': 101,
 'fives': 0,
 'sixes': 7,
 'total_balls_faced': 9586.0,
 'fours_per_ball': 28.389999999999993,
 'sixes_per_ball': 1.3,
 'dots_per_ball': 31.12}

In [1]:
cut_primary_kw = [
]
cut_search_kw = [
    'cut',
    ('point', 0.2),
    ('cover', 0.2),
    ('third man', 0.2),
    ('short', 0.2),
    ('wide', 0.2)
    ('bouncer', 0.2),
    ('bumper', 0.3),
    
]
cut_exlude_words = [
    'full', 'pull', 'hook', 'flick', 'drive', ('leg', 0.3), ('pads', 0.3), ('cut'),
    'lets one go', 'easy leave', 'leaves the ball', 'left alone', 'pitching', 'lets it go', 'inside edge', ('cutter', 0.3)
]

In [11]:
cut_ind, cut_scores = af.search_for_phrases(kohli_comms_flat, cut_search_kw, cut_exlude_words, primary_keywords=cut_primary_kw, threshold=0.49)
cuts_career = [(kohli_comms_flat[i],i) for i,x in enumerate(cut_ind) if x==1]
cuts_almost = [(kohli_comms_flat[i],i) for i,x in enumerate(cut_scores) if x < 0.49 and x > 0]

In [12]:
cuts_career

[('Impatient little shot, goes for the cut, the ball bounces at him, takes the top edge but clears the infield',
  21),
 ('Bouncer, too short really, easy for kohli to duck under', 34),
 ('Good bouncer, in the chest-throat area, fended past short leg', 41),
 ('Another bouncer, too short though and not the best line either. well outside off and kohli gets under it.',
  67),
 ('Nervy moments for kohli. survives another bouncer without much control. bouncer heading for the ribs, he hops back and tries to ride the bounce but ends up limply providing the handle of the bat as his last line of defence. it could have gone anywhere, but managed to stay just away from barath at short leg.',
  83),
 ("It's that kicker that bishoo bowls now and again outside off, looking for the uppish cut. kohli manages to keep it down towards point.",
  90),
 ('Short ball, kohli cuts without intent towards covers.', 161),
 ('Goes back to one turning away and bouncing, and cuts hard but cannot beat the infield',
